# Severity Index Assessment for the Southwest United States during Hurricane Season
### Authors: Ellie Rafalski and Landon Allmaras
### Contacts: lda5159@psu.edu and whatever ellies email is 

![image](ms3-1.PNG)
![image2](ms3-2.PNG)

#### This project presents a potential methodology for calculating a 'Hurricane Severity Index'. We have gathered a few different ECMWF model runs that range from a low risk tropical storm to a hurricane that can cause major damage. Along with that, bouncing between different variables that include wind speed, surface pressure, precipitation can prove helpful to emergency planners, utility managers, and insurers in assessing the potential impact of hurricanes and tropical storms.

## How may a potential employer use this?
1. Track progress of a storm and where damage could be higher at.
2. Predict future trends where storm damage is higher at.
3. Informing the general public when the proper time to evacuate and come back is.
4. Support possible insurance claims that are made after a storm.
5. Prioritize high risk areas for utility restoration.

![damage](damage473.jpg)

## **Choosing which data to use and how to navigate downloading it:**

In [ ]:
from herbie import Herbie, FastHerbie
import pandas as pd, numpy as np
import xarray as xr
import dask

runtime = pd.Timestamp("2024-09-27", tz="utc").replace(tzinfo=None).floor('24h')
H = Herbie(runtime, model="ifs", product="oper", fxx=6, save_dir='.', overwrite=True)

- Using Herbie is a helpful tool when trying to pull any old runs that you may be interested in
- This is also a single run - in this case it will start at 00 Z and take the 6 hour forecast run
- Out of this single run we defined which variables we wanted to use that we believed could be helpful for putting together our 'Severity Index'

### Vorticity, Surface Pressure, Total Wind, and Precipitation rate
- Defining graphs for each of our variables gave us an idea of what could be a crucial part of the calculation and watching the progress of the run.
- We also combined a few different contour and contourf values to read different components on the projection.

Image of combined wind components and temperature:
![image4](ms3-4.png)
Image of surface pressure and precipitation:
![image5](ms3-5.png)

## Defining a full model run for 36 forecasting hours:

In [ ]:
H = FastHerbie([runtime], model="ifs", product="oper", fxx=np.arange(0,246,6).tolist(), save_dir='.', overwrite=True)

- FastHerbie was what we used following our original run and combination of graphs for a single forecasting hour.
- Attributes within FastHerbie help make a full run from 0 to 246 run more smoothly

## Following our download of a full model run... we defined our equation for the Severity index:

In [ ]:
severity_index = abs((0.5 * totalwind_norm) + (0.3 * prate_norm) + (0.15 * vorticity_norm) + (0.05 * pressure_norm))
severity_index = severity_index * 10

- With the variables we had stated previously, we built a **Severity Index** in which the most contributing components were **Wind** and **Precipitation Rate**, based off the fact that these parameters are recorded the most and necessarily are more important when relating to damage that is recorded after a storm. **Vorticity** and **Surface Pressure** are also a smaller piece of the equation, however their values seemed to alter the number of our desired interest from 1-5 for our defined interest.
- After developing the formula for our index, we then developed another formula that combined all of the previously defined functions above. It has the capability of taking a full model run, and outputting the severity index values for that run, as well as all of the normalized variables for that specific run. We decided to then multiply the entire index by 10, just to make the numbers make a little bit more sense on our defined index 0-5.

In [ ]:
def find_threat(lst):
     # pulling in the previously defined functions and use them on the inputted list
    pressure_norm = find_pressure(lst)
    vorticity_norm = find_vorticity(lst)
    totalwind_norm = find_wind(lst)
    prate_norm = find_prate(lst)

    # threat formula 
    severity_index = abs((0.5 * totalwind_norm) + (0.3 * prate_norm) + (0.15 * vorticity_norm) + (0.05 * pressure_norm))
    # weighted severity index, we decided to have it mostly based on total wind and precipitation, with small components of the vorticity and pressure for some more depth
    severity_index = severity_index * 10
    # multitplied our index by ten to make the values a little more " realistic " to have a range 1-5 instead of decimals
    return severity_index

Now that we introduced our Severity Index, we can make subplots for our variables that were chosen and include the movement of the storm and where it is going to be affected at the most

This image correlates with Hurricane Helene at 06 Z
![image6](ms3-6.png)
![image7](ms3-7.png)

## Creation of plots for a full model run:

## Conclusion:
This tutorial demonstrated how meteorological variables can be synthesized into a Hurricane/Tropical Storm Severity Index, offering a quantifiable means of assessing storm impact. This methodology can be adapted for operational use to improve situational awareness and inform emergency management decision-making during severe weather events.
## Future possibilities:
- Explore different models outside of the ECMWF and what is included within Herbie
- Calibrate with more past events to see if our severity index can match with what has already been recorded.
- Deploy a web dashboard that can be accesses by other professionals